In [1]:
from azureml.core import Workspace, Datastore
from azureml.data.datapath import DataPath
from azureml.core.dataset import Dataset
import os

In [2]:
# Access the workspace
ws = Workspace.from_config()

# Access the datastore
ds = Datastore.get(ws, 'myindepentstudyworkdata')

# Set the local path where you want to download the files
local_path = 'GeneratedImages'  # e.g., 'data'
if not os.path.exists(local_path):
    os.makedirs(local_path)


In [4]:
# List of your folder names in the blob storage
folders = ['Bharat_GeneratedImages/Large_new', 'Bharat_GeneratedImages/Normal_new', 'Bharat_GeneratedImages/Small_new/']

for folder in folders:
    # Define the path in the datastore
    datastore_path = DataPath(ds, folder)
    
    # Define the path in the local workspace
    download_path = os.path.join(local_path, folder)
    
    # Create the local directory if it does not exist
    if not os.path.exists(download_path):
        os.makedirs(download_path)
    
    # Create a dataset pointing to the files in the folder
    dataset = Dataset.File.from_files(datastore_path)
    
    # Download the files in the folder to the local workspace
    dataset.download(target_path=download_path, overwrite=True)
    print(f"Files in {folder} have been downloaded to {download_path}")


{'infer_column_types': 'False', 'activity': 'download'}
{'infer_column_types': 'False', 'activity': 'download', 'activityApp': 'FileDataset'}
Files in Bharat_GeneratedImages/Large_new have been downloaded to GeneratedImages/Bharat_GeneratedImages/Large_new
{'infer_column_types': 'False', 'activity': 'download'}
{'infer_column_types': 'False', 'activity': 'download', 'activityApp': 'FileDataset'}
Files in Bharat_GeneratedImages/Normal_new have been downloaded to GeneratedImages/Bharat_GeneratedImages/Normal_new
{'infer_column_types': 'False', 'activity': 'download'}
{'infer_column_types': 'False', 'activity': 'download', 'activityApp': 'FileDataset'}
Files in Bharat_GeneratedImages/Small_new/ have been downloaded to GeneratedImages/Bharat_GeneratedImages/Small_new/


In [3]:
import numpy as np
from scipy.linalg import sqrtm
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.preprocessing.image import load_img, img_to_array
from keras.models import Model
import glob

2024-04-01 23:11:13.020396: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-01 23:11:16.255689: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
def load_images(path, size=(299, 299)):
    images_list = []
    for filename in glob.glob(path + '/*.png'):  # assuming jpg format
        image = load_img(filename, target_size=size)
        image = img_to_array(image)
        # Expand dimensions to fit model expected input
        image = np.expand_dims(image, axis=0)
        images_list.append(image)
    images = np.vstack(images_list)
    return images

def preprocess_images(images):
    images = images.astype('float32')
    images = preprocess_input(images)  # Preprocess based on InceptionV3 requirements
    return images

In [5]:
def model_feature_extractor():
    model = InceptionV3(include_top=False, pooling='avg', input_shape=(299, 299, 3))
    return model

def get_image_features(model, images):
    features = model.predict(images, verbose=0)
    return features


In [6]:
def calculate_fid(real_features, generated_features):
    mu1, sigma1 = real_features.mean(axis=0), np.cov(real_features, rowvar=False)
    mu2, sigma2 = generated_features.mean(axis=0), np.cov(generated_features, rowvar=False)
    ssdiff = np.sum((mu1 - mu2)**2.0)
    covmean, _ = sqrtm(sigma1.dot(sigma2), disp=False)
    if np.iscomplexobj(covmean):
        covmean = covmean.real
    # Numerical stability fix
    sigma1 += np.eye(sigma1.shape[0]) * 1e-6
    sigma2 += np.eye(sigma2.shape[0]) * 1e-6
    fid = ssdiff + np.trace(sigma1 + sigma2 - 2.0 * sqrtm(sigma1.dot(sigma2)))
    if np.iscomplexobj(fid):
        fid = fid.real
    return fid


In [7]:
!pwd

/mnt/batch/tasks/shared/LS_root/mounts/clusters/bkathuri1/code/Users/bkathuri


In [8]:
# Paths to your datasets
real_images_path = 'Train/Small'  # Update this path
generated_images_path = 'GeneratedImages/Bharat_GeneratedImages/Small_new'  # Update this path

# Load and preprocess images
real_images = preprocess_images(load_images(real_images_path))
generated_images = preprocess_images(load_images(generated_images_path))

# Load the model
model = model_feature_extractor()

# Extract features
real_features = get_image_features(model, real_images)
generated_features = get_image_features(model, generated_images)

# Calculate FID
fid = calculate_fid(real_features, generated_features)
print(f'FID score: {fid}')


2024-04-01 23:13:39.967040: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


FID score: 86.29920753459524


In [9]:
# Paths to your datasets
real_images_path = 'Train/Normal'  # Update this path
generated_images_path = 'GeneratedImages/Bharat_GeneratedImages/Normal_new'  # Update this path

# Load and preprocess images
real_images = preprocess_images(load_images(real_images_path))
generated_images = preprocess_images(load_images(generated_images_path))

# Load the model
model = model_feature_extractor()

# Extract features
real_features = get_image_features(model, real_images)
generated_features = get_image_features(model, generated_images)

# Calculate FID
fid = calculate_fid(real_features, generated_features)
print(f'FID score: {fid}')


FID score: 60.38588786532444


In [10]:
# Paths to your datasets
real_images_path = 'Train/Large'  # Update this path
generated_images_path = 'GeneratedImages/Bharat_GeneratedImages/Large_new'  # Update this path

# Load and preprocess images
real_images = preprocess_images(load_images(real_images_path))
generated_images = preprocess_images(load_images(generated_images_path))

# Load the model
model = model_feature_extractor()

# Extract features
real_features = get_image_features(model, real_images)
generated_features = get_image_features(model, generated_images)

# Calculate FID
fid = calculate_fid(real_features, generated_features)
print(f'FID score: {fid}')


FID score: 54.7671899856191
